## Schema 1: Intensive care unit stay

![Cube 1](img/Cube1.png)

In [7]:
require 'sequel'

DB_MIMIC = Sequel.connect(adapter: 'postgres',
                          host: 'postgres',
                          user: 'postgres',
                          password: 'password',
                          database: 'mimic3',
                          search_path: ['mimiciii']
                          )

DB_OLAP = Sequel.connect(adapter: 'postgres',
                          host: 'postgres',
                          user: 'postgres',
                          password: 'password',
                          database: 'mimic3',
                          search_path: ['olap']
                          )

load '../etl/model.rb'
nil

<main>:2: warning: already initialized constant DB_MIMIC
<main>:2: warning: previous definition of DB_MIMIC was here
<main>:10: warning: already initialized constant DB_OLAP
<main>:10: warning: previous definition of DB_OLAP was here


#### Time dimension table

In [9]:
# Creation
DB_OLAP.drop_table?(:icu_stay_time)
DB_OLAP.create_table(:icu_stay_time) do
  String :day, primary_key: true
  String :month
  String :year
end

# Population
Icustay.all.uniq.each do |icustay|
  time = icustay.intime
  year = time.year
  month = time.month
  day = time.day
  day_s = "#{year}-#{month}-#{day}"
  if DB_OLAP[:icu_stay_time].where(:day => day_s).empty?
    DB_OLAP[:icu_stay_time].insert(
      day: day_s,
      month: "#{year}-#{month}",
      year: year.to_s
    )
  end
end

nil

#### Patient age dimension table

In [10]:
# Method for calculating a patient's age using the difference between birth year and ICU stay year.
# If the birthday did not occur yet by the time of ICU stay then we subtract a year from the age.
def age_in_completed_years (bd, d)
    a = d.year - bd.year
    a = a - 1 if (
         bd.month >  d.month or 
        (bd.month >= d.month and bd.day > d.day)
    )
    a
end

:age_in_completed_years

In [11]:
# Creation
DB_OLAP.drop_table?(:icu_stay_pat_age)
DB_OLAP.create_table(:icu_stay_pat_age) do
  Integer :age_id, primary_key: true
  String :age
  String :decade
end

# Population
Icustay.all.uniq.each do |icustay|
  patient = icustay.patient
  age = age_in_completed_years(patient.dob, icustay.intime)
  decade = age / 10
  decade_str = "#{decade}0 - #{decade}9"  
  if DB_OLAP[:icu_stay_pat_age].where(age_id: age).empty?
    DB_OLAP[:icu_stay_pat_age].insert(
      age_id: age,
      age: age.to_s,
      decade: decade_str
    )
  end
end

nil

#### Death dimension table

In [12]:
#Creation
DB_OLAP.drop_table?(:icu_stay_deaths)
DB_OLAP.create_table(:icu_stay_deaths) do
  Integer :hospital_expire_flag, primary_key: true
  String :death
end

# Population
DB_OLAP[:icu_stay_deaths].insert(
  hospital_expire_flag: 0,
  death: 'alive'
)
DB_OLAP[:icu_stay_deaths].insert(
  hospital_expire_flag: 1,
  death: 'dead'
)


nil

#### Fact table

In [14]:
# Creation
DB_OLAP.drop_table?(:icustays)
DB_OLAP.create_table(:icustays) do
  Integer :id, primary_key: true
  # Measures
  Integer :elapsed_days
  Integer :diagnoses_count
  # Dimension keys 
  String :day
  Integer :hospital_expire_flag
  Integer :age_id
end

# Population
Icustay.all.uniq.each do |icustay|
  
  days = nil
  if not icustay.outtime.nil?
    days = (icustay.outtime - icustay.intime).to_i / ( 3600 * 24 )  # Calculte elapsed days
  end
  
  DB_OLAP[:icustays].insert(
    id: icustay.row_id,
    
    elapsed_days: days,
    diagnoses_count: icustay.admission.diagnosis.nil? ? 0 : icustay.admission.diagnosis.length,
    
    hospital_expire_flag: icustay.admission.hospital_expire_flag,
    day: "#{icustay.intime.year}-#{icustay.intime.month}-#{icustay.intime.day}",
    age_id: age_in_completed_years(icustay.patient.dob, icustay.intime)
  )
end

nil

## XML implementation

## TESTS

In [48]:
DB_OLAP[:icu_stay_pat_age].where(age_id: 5).empty?

true

In [77]:
a = Icustay.first.intime
b = Icustay.first.patient.dob
age_in_completed_years(b, a)

70

In [26]:
icustay = Icustay.first
DateTime.strptime((icustay.outtime - icustay.intime).to_s, '%s')
#Icustay.each do |icustay| 
#  puts "#{icustay.row_id}: #{icustay.admission.diagnosis}"
#end
Icustay[12763]
DiagnoseIcd.where(:hadm_id => Icustay[12763].hadm_id).all

[#<DiagnoseIcd @values={:row_id=>112513, :subject_id=>10027, :hadm_id=>199395, :seq_num=>1, :icd9_code=>"4280"}>, #<DiagnoseIcd @values={:row_id=>112514, :subject_id=>10027, :hadm_id=>199395, :seq_num=>2, :icd9_code=>"4240"}>, #<DiagnoseIcd @values={:row_id=>112515, :subject_id=>10027, :hadm_id=>199395, :seq_num=>3, :icd9_code=>"3970"}>, #<DiagnoseIcd @values={:row_id=>112516, :subject_id=>10027, :hadm_id=>199395, :seq_num=>4, :icd9_code=>"4266"}>, #<DiagnoseIcd @values={:row_id=>112517, :subject_id=>10027, :hadm_id=>199395, :seq_num=>5, :icd9_code=>"42731"}>, #<DiagnoseIcd @values={:row_id=>112518, :subject_id=>10027, :hadm_id=>199395, :seq_num=>6, :icd9_code=>"5859"}>, #<DiagnoseIcd @values={:row_id=>112519, :subject_id=>10027, :hadm_id=>199395, :seq_num=>7, :icd9_code=>"41401"}>, #<DiagnoseIcd @values={:row_id=>112520, :subject_id=>10027, :hadm_id=>199395, :seq_num=>8, :icd9_code=>"4019"}>, #<DiagnoseIcd @values={:row_id=>112521, :subject_id=>10027, :hadm_id=>199395, :seq_num=>9, :i